In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from openpyxl.drawing.image import Image
from io import BytesIO
from math import sin, cos, exp, sqrt, pi

In [7]:
# Evaluation functions
def f1(x: np.ndarray) -> float:
    """
    Evaluation function f1.

    Args:
        x (np.ndarray): Input array representing an individual.

    Returns:
        float: Fitness value for f1.
    """
    return -5 / (1 + np.sum(x ** 2)) + sin(1 / np.tan(exp(-5 / (1 + np.sum(x ** 2)))))

def f2(x: np.ndarray, a: float = 20, b: float = 0.2, c: float = 2 * pi) -> float:
    """
    Evaluation function f2 (Ackley function).

    Args:
        x (np.ndarray): Input array representing an individual.
        a (float): Parameter a for the Ackley function.
        b (float): Parameter b for the Ackley function.
        c (float): Parameter c for the Ackley function.

    Returns:
        float: Fitness value for f2.
    """
    n = len(x)
    sum_sq_term = -a * exp(-b * sqrt(np.sum(x ** 2) / n))
    cos_term = -exp(np.sum(np.cos(c * x)) / n)
    return sum_sq_term + cos_term + a + exp(1)

In [8]:
def binary_to_real(binary_individual: np.ndarray, 
                   domain: Tuple[float, float], dimensions: int) -> np.ndarray:
    """
    Convert binary individual to real-valued individual.

    Args:
        binary_individual (np.ndarray): Binary array representing an individual.
        domain (Tuple[float, float]): Domain bounds (min, max) for each variable.
        dimensions (int): Number of dimensions (variables).

    Returns:
        np.ndarray: Real-valued representation of the individual.
    """
    max_value = 2 ** 16 - 1  # Max val for 16 bits
    real_individual = []
    for i in range(dimensions):
        binary_value = int("".join(map(str, binary_individual[i*16:(i+1)*16])), 2)
        real_value = domain[0] + (binary_value / max_value) * (domain[1] - domain[0])
        real_individual.append(real_value)
    return np.array(real_individual)

In [9]:
def initialize_population(pop_size: int, dimensions: int, 
                          domain: Tuple[float, float], binary: bool = False) -> np.ndarray:
    """
    Initialize a population of individuals.

    Args:
        pop_size (int): Number of individuals in the population.
        dimensions (int): Number of dimensions for each individual.
        domain (Tuple[float, float]): Domain bounds (min, max) for each variable.
        binary (bool): Whether to initialize in binary or real-valued format.

    Returns:
        np.ndarray: Initialized population.
    """
    if binary:
        return np.random.randint(2, size=(pop_size, dimensions * 16))  # Binary
    return np.random.uniform(domain[0], domain[1], (pop_size, dimensions)) # Decimal

In [5]:
def tournament_selection(pop: np.ndarray, scores: np.ndarray, 
                         k: int = 3) -> np.ndarray:
    """
    Select an individual from the population using tournament selection.

    Args:
        pop (np.ndarray): Population array.
        scores (np.ndarray): Fitness scores for the population.
        k (int): Tournament size.

    Returns:
        np.ndarray: Selected individual.
    """
    selected = np.random.choice(range(len(pop)), k)
    best = selected[np.argmin([scores[i] for i in selected])]
    return pop[best]

In [6]:
def crossover(parent1: np.ndarray, parent2: np.ndarray, 
              binary: bool = False) -> Tuple[np.ndarray, np.ndarray]:
    """
    Perform crossover operation between two parents to produce offspring.

    Args:
        parent1 (np.ndarray): First parent.
        parent2 (np.ndarray): Second parent.
        binary (bool): Whether to perform binary or real-valued crossover.

    Returns:
        Tuple[np.ndarray, np.ndarray]: Two offspring individuals.
    """
    if binary:
        point = np.random.randint(1, len(parent1) - 1)
        child1 = np.concatenate((parent1[:point], parent2[point:]))
        child2 = np.concatenate((parent2[:point], parent1[point:]))
    else:
        alpha = np.random.rand()
        child1 = alpha * parent1 + (1 - alpha) * parent2
        child2 = alpha * parent2 + (1 - alpha) * parent1
    return child1, child2

In [7]:
def mutate(individual: np.ndarray, mutation_rate: float = 0.01, 
           binary: bool = False, domain: Tuple[float, float] = (-5, 5)) -> np.ndarray:
    """
    Apply mutation to an individual.

    Args:
        individual (np.ndarray): The individual to be mutated.
        mutation_rate (float): Probability of mutation for each gene.
        binary (bool): Whether the individual is in binary format.
        domain (Tuple[float, float]): Domain bounds for real-valued mutation.

    Returns:
        np.ndarray: Mutated individual.
    """
    if binary:
        for i in range(len(individual)):
            if np.random.rand() < mutation_rate:
                individual[i] = 1 - individual[i]
    else:
        individual += np.random.normal(0, 0.1, size=individual.shape)
        individual = np.clip(individual, domain[0], domain[1])
    return individual

In [8]:
def evolutionary_algorithm(pop_size: int, dimensions: int, domain: Tuple[float, float], 
                           generations: int, eval_func: Callable[[np.ndarray], float], 
                           binary: bool = False, max_evaluations: int = 10000) -> List[float]:
    """
    Run the evolutionary algorithm for optimization.

    Args:
        pop_size (int): Population size.
        dimensions (int): Number of dimensions (variables) for each individual.
        domain (Tuple[float, float]): Domain bounds for the problem.
        generations (int): Number of generations to run the algorithm.
        eval_func (Callable[[np.ndarray], float]): Evaluation function for fitness.
        binary (bool): Whether to use binary representation.
        max_evaluations (int): Maximum number of evaluations allowed.

    Returns:
        List[float]: List of best scores per generation.
    """
    pop = initialize_population(pop_size, dimensions, domain, binary)
    scores = []
    best_scores = []
    eval_count = 0

    while eval_count < max_evaluations:
        # Decoding binary
        if binary:
            decoded_pop = [binary_to_real(ind, domain, dimensions) for ind in pop]
            scores = np.array([eval_func(ind) for ind in decoded_pop])
        else:
            scores = np.array([eval_func(ind) for ind in pop])

        best = np.min(scores)
        best_scores.append(best)
        eval_count += len(scores)

        # Creating new population
        new_pop = []
        for _ in range(pop_size // 2):
            parent1 = tournament_selection(pop, scores)
            parent2 = tournament_selection(pop, scores)
            child1, child2 = crossover(parent1, parent2, binary)
            new_pop.append(mutate(child1, binary=binary, domain=domain))
            new_pop.append(mutate(child2, binary=binary, domain=domain))
        pop = np.array(new_pop)

    return best_scores

In [9]:
def run_experiments() -> dict:
    """
    Run multiple experiments with different configurations.

    Returns:
        dict: Dictionary with experiment results for each configuration.
    """
    results = {}
    results['F1_real'] = [evolutionary_algorithm(pop_size, dimensions, domain_f1, generations, f1, binary=False) for _ in range(100)]
    results['F2_real'] = [evolutionary_algorithm(pop_size, dimensions, domain_f2, generations, f2, binary=False) for _ in range(100)]
    results['F1_binary'] = [evolutionary_algorithm(pop_size, dimensions, domain_f1, generations, f1, binary=True) for _ in range(100)]
    results['F2_binary'] = [evolutionary_algorithm(pop_size, dimensions, domain_f2, generations, f2, binary=True) for _ in range(100)]
    return results

In [10]:
def save_results_to_excel(results: dict) -> None:
    """
    Save experiment results to an Excel file, including plots.

    Args:
        results (dict): Dictionary with experiment results.
    """
    with pd.ExcelWriter("results.xlsx", engine="openpyxl") as writer:
        for name, data in results.items():
            data = np.array(data)
            df_raw = pd.DataFrame(data)
            df_raw.to_excel(writer, sheet_name=f'{name}_raw', index=False)

            mean_data = np.mean(data, axis=0)
            df_mean = pd.DataFrame(mean_data, columns=['Mean'])
            df_mean.to_excel(writer, sheet_name=f'{name}_mean', index=False)

            plt.figure()
            plt.plot(mean_data, label=f'Average Best Score ({name})')
            plt.xlabel("Generation")
            plt.ylabel("Fitness")
            plt.legend()
            plt.title(f'Average Best Scores - {name}')

            img_data = BytesIO()
            plt.savefig(img_data, format='png')
            img_data.seek(0)
            plt.close()

            workbook = writer.book
            worksheet = workbook[f'{name}_mean']
            img = Image(img_data)
            worksheet.add_image(img, 'E5')

In [4]:
# Experiment variables
pop_size = 50
dimensions = 10
domain_f1 = (-3, 3)
domain_f2 = (-32.768, 32.768)
generations = 200

In [12]:
# Run experiments and save results
results = run_experiments()
save_results_to_excel(results)